In [175]:
from DbConnector import DbConnector
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm

connection = DbConnector()
db_connection = connection.db_connection
cursor = connection.cursor

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assigment2_store_distribuerte',)
-----------------------------------------------



#### Task 2

In [158]:
"""
1. How many users, activities and trackpoints are there in the dataset (after it is
inserted into the database).
"""

num_users_query = """SELECT COUNT(*) FROM User"""
cursor.execute(num_users_query)
num_users = cursor.fetchone()[0]
print(f'Number of users in the dataset: {num_users}')

num_activities_query = """SELECT COUNT(*) FROM Activity"""
cursor.execute(num_activities_query)
num_activities = cursor.fetchone()[0]
print(f'Number of activities in the dataset: {num_activities}')

num_trackpoints_query = """SELECT COUNT(*) FROM TrackPoint"""
cursor.execute(num_trackpoints_query)
num_trackpoints = cursor.fetchone()[0]
print(f'Number of trackpoints in the dataset: {num_trackpoints}')

Number of users in the dataset: 182
Number of activities in the dataset: 16048
Number of trackpoints in the dataset: 9681756


In [159]:
"""
2. Find the average, maximum and minimum number of trackpoints per user.
"""

avg_max_min_per_user_query = """
SELECT AVG(trackpoint_count), MAX(trackpoint_count), MIN(trackpoint_count)
FROM (
    SELECT U.id AS user_id, COUNT(T.id) AS trackpoint_count
    FROM User U
    LEFT JOIN Activity A ON U.id = A.user_id
    LEFT JOIN TrackPoint T ON A.id = T.activity_id
    GROUP BY U.id
) AS UserTrackpointCounts
"""

cursor.execute(avg_max_min_per_user_query)
avg_max_min_per_user = cursor.fetchall()
print(f'Average number of trackpoints per user: {avg_max_min_per_user[0][0]}')
print(f'Max number of trackpoints per user: {avg_max_min_per_user[0][1]}')
print(f'Min number of trackpoints per user: {avg_max_min_per_user[0][2]}')

Average number of trackpoints per user: 53196.4615
Max number of trackpoints per user: 1010325
Min number of trackpoints per user: 0


In [160]:
"""
3. Find the top 15 users with the highest number of activities.
"""

top15_users_query = """
SELECT U.id, COUNT(A.id) AS ActivityCount
FROM User U
INNER JOIN Activity A ON U.id = A.user_id
GROUP BY U.id
ORDER BY ActivityCount DESC
LIMIT 15;
"""

cursor.execute(top15_users_query)
top15_users = cursor.fetchall()
for users in top15_users:
    print(f'User {users[0]}  \tActivity count {users[1]}')

User 128  	Activity count 2102
User 153  	Activity count 1793
User 25  	Activity count 715
User 163  	Activity count 704
User 62  	Activity count 691
User 144  	Activity count 563
User 41  	Activity count 399
User 85  	Activity count 364
User 4  	Activity count 346
User 140  	Activity count 345
User 167  	Activity count 320
User 68  	Activity count 280
User 17  	Activity count 265
User 3  	Activity count 261
User 14  	Activity count 236


In [161]:
"""
4. Find all users who have taken a bus.
"""

bus_users_query = """
SELECT U.id
FROM User U
INNER JOIN Activity A ON U.id = A.user_id
WHERE A.transportation_mode = "Bus"
GROUP BY U.id
"""

cursor.execute(bus_users_query)
bus_users = cursor.fetchall()
bussers = [user[0] for user in bus_users]
print(f'Bus users: {bussers}')

Bus users: ['10', '52', '62', '73', '81', '84', '85', '91', '92', '112', '125', '128', '175']


In [162]:
"""
5. List the top 10 users by their amount of different transportation modes.
"""

nunique_transportation_mode_query = """
SELECT U.id, COUNT(DISTINCT A.transportation_mode) AS unique_transportation_mode
FROM User U
INNER JOIN Activity A on U.id = A.user_id
GROUP BY U.id
ORDER BY unique_transportation_mode DESC
LIMIT 10
"""

cursor.execute(nunique_transportation_mode_query)
nunique_transportation_mode = cursor.fetchall()

for users in nunique_transportation_mode:
    print(f'User {users[0]}  Transportation mode count {users[1]}')

User 128  Transportation mode count 10
User 62  Transportation mode count 8
User 85  Transportation mode count 5
User 84  Transportation mode count 4
User 163  Transportation mode count 4
User 112  Transportation mode count 4
User 81  Transportation mode count 4
User 78  Transportation mode count 4
User 58  Transportation mode count 4
User 102  Transportation mode count 3


In [163]:
"""
6. Find activities that are registered multiple times. You should find the query even
if it gives zero result.
"""

duplicate_activities_query = """
SELECT user_id, transportation_mode, start_date_time, end_date_time, COUNT(*) AS occurrence_count
FROM Activity
GROUP BY user_id, transportation_mode, start_date_time, end_date_time
HAVING COUNT(*) > 1;
"""

cursor.execute(duplicate_activities_query)
duplicate_activities = cursor.fetchall()
duplicate_activities

[]

In [165]:
"""
7. a) Find the number of users that have started an activity in one day and ended
the activity the next day.
"""

activity_time_query = """
SELECT A.user_id, A.id, A.transportation_mode, A.start_date_time, A.end_date_time
FROM User U
INNER JOIN Activity A ON U.id = A.user_id
"""

df = pd.read_sql_query(activity_time_query, db_connection)
df["time_duration"] = df.end_date_time - df.start_date_time
df = df[df["time_duration"] >= np.timedelta64(1, 'D')]

print(f'Number of users which had activities longer than 1 day: {df["user_id"].nunique()}')

"""
7. b) List the transportation mode, user id and duration for these activities
"""

df[['transportation_mode', 'user_id', 'time_duration']]

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_8716\417328831.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(activity_time_query, db_connection)


Number of users which had activities longer than 1 day: 5


,transportation_mode,user_id,time_duration
4914,other,144,1 days 04:28:34
4920,other,144,1 days 02:28:35
8955,other,17,1 days 00:32:16
10489,other,28,1 days 05:12:45
12831,other,51,1 days 05:01:03
16041,other,99,1 days 04:49:19


In [154]:
"""
8. Find the number of users which have been close to each other in time and space.
Close is defined as the same space (50 meters) and for the same half minute (30
seconds)
"""

proximity_query = """
SELECT A.user_id, A.id, A.start_date_time, A.end_date_time, T.lat, T.lon
FROM Activity A
INNER JOIN TrackPoint T ON A.id = T.activity_id
"""

df = pd.read_sql_query(proximity_query, db_connection)
df

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_8716\2688048494.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(proximity_query, db_connection)


KeyboardInterrupt: 

In [ ]:
# Precompute constants
R = 6371000
lat_rad = np.radians(df['lat'])
lon_rad = np.radians(df['lon'])
time_threshold = timedelta(seconds=30)
distance_threshold = 50

user_pairs = []

# Group data by user_id
grouped = df.groupby('user_id')

# Iterate through each group (user)
for user_id, group_data in tqdm(grouped):
    # Iterate through rows in the group
    for i in range(len(group_data)):
        for j in range(i + 1, len(group_data)):
            row1 = group_data.iloc[i]
            row2 = group_data.iloc[j]

            # Calculate Haversine distance
            dlat = lat_rad[j] - lat_rad[i]
            dlon = lon_rad[j] - lon_rad[i]
            a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_rad[i]) * np.cos(lat_rad[j]) * np.sin(dlon / 2.0) ** 2
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
            meters = R * c

            # Calculate time difference
            time_diff = row2['start_date_time'] - row1['start_date_time']

            # Check criteria (30 seconds and 50 meters)
            if time_diff <= time_threshold and meters <= distance_threshold:
                user_pairs.append((user_id, row1['id'], row2['id']))

# Convert the result to a DataFrame
result_df = pd.DataFrame(user_pairs, columns=['user_id', 'id1', 'id2'])

result_df

  0%|          | 0/173 [03:08<?, ?it/s]


KeyboardInterrupt: 

In [166]:
"""
9. Find the top 15 users who have gained the most altitude meters.
"""

altitude_gainers_query = """
SELECT Activity.user_id, SUM(gained_altitude_activity) * 0.3048 AS gained_altitude_total
FROM (
    SELECT TrackPoint.activity_id AS activity_id,
        GREATEST(TrackPoint.altitude - LAG(TrackPoint.altitude) OVER (ORDER BY TrackPoint.id), 0) AS gained_altitude_activity
    FROM TrackPoint
    WHERE altitude != -777
) AS t
LEFT JOIN Activity ON (t.activity_id = Activity.id)
GROUP BY Activity.user_id
ORDER BY gained_altitude_total DESC
LIMIT 15;
"""

cursor.execute(altitude_gainers_query)
altitude_gainers = cursor.fetchall()

for users in altitude_gainers:
    print(f'User {users[0]}  \tAltitude gained: {users[1]}m')

User 128  	Altitude gained: 688280.7672m
User 153  	Altitude gained: 675336.5208m
User 4  	Altitude gained: 357835.5048m
User 41  	Altitude gained: 280401.9792m
User 62  	Altitude gained: 258156.1512m
User 3  	Altitude gained: 249477.2760m
User 163  	Altitude gained: 246217.7448m
User 85  	Altitude gained: 239366.4504m
User 144  	Altitude gained: 224671.4328m
User 30  	Altitude gained: 183820.0032m
User 39  	Altitude gained: 161123.3760m
User 25  	Altitude gained: 149747.9352m
User 84  	Altitude gained: 145680.6840m
User 167  	Altitude gained: 130406.5464m
User 0  	Altitude gained: 130186.1760m


In [167]:
"""
10.Find the users that have traveled the longest total distance in one day for each
transportation mode.
"""

activity_time_query = """
SELECT A.user_id, A.id, A.transportation_mode, A.start_date_time, A.end_date_time, T.lat, T.lon
FROM Activity A
INNER JOIN TrackPoint T ON A.id = T.activity_id
"""

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of the Earth in meters
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    meters = R * c
    return meters

df = pd.read_sql_query(activity_time_query, db_connection)
df["time_duration"] = df.end_date_time - df.start_date_time
df = df[df["time_duration"] <= np.timedelta64(1, 'D')]

df.sort_values(by=['user_id', 'id', 'start_date_time'], inplace=True)
grouped = df.groupby(['user_id', 'id', 'transportation_mode'])
result_df = grouped.apply(lambda group: haversine(
    group['lat'].iloc[0],
    group['lon'].iloc[0],
    group['lat'].iloc[-1],
    group['lon'].iloc[-1]
)).reset_index(name='distance')
result_df

max_distance_idx = result_df.groupby(['transportation_mode'])['distance'].idxmax()
max_distances = result_df.loc[max_distance_idx, ['user_id', 'transportation_mode', 'distance']].sort_values(by="distance", ascending=False)
max_distances

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_8716\3338472784.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(activity_time_query, db_connection)


,user_id,transportation_mode,distance
4736,142,other,8.688100e+06
2343,128,airplane,4.307009e+06
13320,62,train,2.553725e+05
2030,128,car,2.090882e+05
2273,128,bus,1.469377e+05
2137,128,boat,6.061358e+04
1874,128,bike,3.474795e+04
583,108,walk,2.219934e+04
2327,128,taxi,1.995376e+04
2089,128,subway,1.604496e+04


In [194]:
"""
11.Find all users who have invalid activities, and the number of invalid activities per
# user

An invalid activity is defined as an activity with consecutive trackpoints
 where the timestamps deviate with at least 5 minutes.
"""

invalid_query = """
SELECT user_id, id, dt, diff_seconds
FROM (
    SELECT A.user_id, A.id, T.date_time AS dt,
        TIMESTAMPDIFF(SECOND, lag(T.date_time) OVER (PARTITION BY T.activity_id ORDER BY T.date_time), T.date_time) AS diff_seconds
    FROM Activity A
    INNER JOIN TrackPoint T ON A.id = T.activity_id
) AS subquery
WHERE diff_seconds > 300;
"""

invalid = pd.read_sql_query(invalid_query, db_connection)
display(invalid.groupby(['user_id'])['id'].nunique().reset_index().sort_values(by='user_id'))

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_8716\703956840.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalid = pd.read_sql_query(invalid_query, db_connection)


,user_id,id
0,0,101
1,1,45
2,10,50
3,100,3
4,101,46
...,...,...
166,95,4
167,96,35
168,97,14
169,98,5


In [ ]:
"""
12.Find all users who have registered transportation_mode and their most used
transportation_mode.
    - The answer should be on format (user_id,most_used_transportation_mode) sorted on user_id.
    - Some users may have the same number of activities tagged with e.g. walk 
    and car. In this case it is up to you to decide which transportation mode
    to include in your answer (choose one)
    - Do not count the rows where the mode is null.
"""
